In [1]:
# Playgroud Series - Binary Classification of Insurance Cross Selling
  # https://www.kaggle.com/competitions/playground-series-s4e7
  # Your Goal: predict which customers respond positively to an automobile insurance offer.
  # Metric: Submissions are evaluated using area under the ROC curve.

# Improvements (to do)

In [1]:
#DONE v1 and v2:
# use more data (used 100k on train).  -> DT was practically not improved, even with 1.5M rows on test. Saw an post showing there are many repeated rows on dataset.
    # There are no duplicated rows on test data (11M). Removind 1 feature per time, all of them have many duplicates (downsampling). There was no gain of doing that. (https://www.kaggle.com/competitions/playground-series-s4e7/discussion/517167)
# Plot ROC_AUC curve to compare moodel

#DONE V3:
#Probabilistic approach

#DONE V4:
# tested LogisticRegressionCV and  LogisticRegression + GridSearchCV (fine tuning). -> RF is still superior.

#DONE V5:
# CatBoost wuthout default params

#DONE V6
# Balance CatBoost -> did not imporve
# CatBoost + GridSearchCV -> did not improve
# CatBoost + GridSearchCV of [AutoML Grand Prix] 1st Place Solution (https://www.kaggle.com/code/rohanrao/automl-grand-prix-1st-place-solution) -> did not improve

#DONE V7
# Data Prep/Feature Engeering and Cat Boost of [AutoML Grand Prix] 1st Place Solution (https://www.kaggle.com/code/rohanrao/automl-grand-prix-1st-place-solution)
    # Did not run with 11Mi train rows locally (few mommory). Tried w/ 100k train and full test (needed to submit).
    # User iterations=1000 instead of 5000, and task_type= 'CPU', cause 'GPU' -> Demands NVidea graphics cards w/ CUDA
    # Undestood StratifiedKFold w/ CatBoost cross validation
    # Kaggle public score: 0.884 (best by far!)

#DONE V8:
# Optimize hiperparameters with Scikit Optimize BayesSearchCV
    # Used BayesSearchCV. Got 0.871 on public. Had 0.884 with StratifiedKFold. Looks like its critic on this kind of problem.

#TO DO V9:
#Try using both BayesSearchCV and StratifiedKFold, to improve performance, with pipelines.

#######################
#IMPROVE EDA
# Follow Kaggle notebooks

# Imports

In [30]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tabulate import tabulate
import polars as pl

pd.set_option('display.float_format', lambda x: '%.3f' % x) #pandas decimal places
pd.set_option('max_colwidth', 100) #num of cols to be shown
np.set_printoptions(suppress=True, precision=6)#numpy decimal placese


from sklearn.model_selection       import train_test_split, GridSearchCV, StratifiedKFold#, cross_val_score
from sklearn.preprocessing         import MinMaxScaler #OneHotEncoder  
from sklearn.metrics               import roc_auc_score, roc_curve, auc, confusion_matrix, ConfusionMatrixDisplay
from sklearn.tree                  import DecisionTreeClassifier, plot_tree
from sklearn.ensemble              import RandomForestClassifier
from sklearn.linear_model          import LogisticRegressionCV, LogisticRegression
from catboost                      import CatBoostClassifier, Pool #Numpy downgrade needed: pip install numpy==1.26.4 (from mar/24)
from skopt                         import BayesSearchCV, space

# #import sweetviz as sv
# from yellowbrick.features import ParallelCoordinates

# Load Data

In [13]:
## load data
#train = pl.read_csv('..\\data\\kaggle\\train.csv')

# Specify the path to your CSV file and read a random sample of the dataset
train = pl.read_csv('..\\data\\kaggle\\train.csv').sample(n=500000)
test = pl.read_csv('..\\data\\kaggle\\test.csv') #.sample(n=70000) needs to be full, cause I neet to submit all the rows!
test = test.with_columns(pl.lit(0).cast(pl.Int64).alias('Response'))

In [14]:
train[:2]

id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
i64,str,i64,i64,f64,i64,str,str,f64,f64,i64,i64
5261264,"""Female""",21,1,47.0,1,"""< 1 Year""","""No""",41835.0,152.0,122,0
11305325,"""Female""",20,1,46.0,1,"""< 1 Year""","""No""",27499.0,160.0,124,0


In [15]:
test[:2]

id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
i64,str,i64,i64,f64,i64,str,str,f64,f64,i64,i64
11504798,"""Female""",20,1,47.0,0,"""< 1 Year""","""No""",2630.0,160.0,228,0
11504799,"""Male""",47,1,28.0,0,"""1-2 Year""","""Yes""",37483.0,124.0,123,0


In [16]:
## prepare data
df = pl.concat([train, test])

df = df.with_columns([
    pl.col('Gender').replace({'Male': 0, 'Female': 1}).cast(pl.Int32),
    pl.col('Region_Code').cast(int),
    pl.col('Vehicle_Age').replace({'< 1 Year': 0, '1-2 Year': 1, '> 2 Years': 2}).cast(pl.Int32),
    pl.col('Vehicle_Damage').replace({'No': 0, 'Yes': 1}).cast(pl.Int32),
    pl.col('Annual_Premium').cast(int),
    pl.col('Policy_Sales_Channel').cast(int)
])

df = df.with_columns([
    (pl.Series(pd.factorize((df['Previously_Insured'].cast(str) + df['Annual_Premium'].cast(str)).to_numpy())[0])).alias('Previously_Insured_Annual_Premium'),
    (pl.Series(pd.factorize((df['Previously_Insured'].cast(str) + df['Vehicle_Age'].cast(str)).to_numpy())[0])).alias('Previously_Insured_Vehicle_Age'),
    (pl.Series(pd.factorize((df['Previously_Insured'].cast(str) + df['Vehicle_Damage'].cast(str)).to_numpy())[0])).alias('Previously_Insured_Vehicle_Damage'),
    (pl.Series(pd.factorize((df['Previously_Insured'].cast(str) + df['Vintage'].cast(str)).to_numpy())[0])).alias('Previously_Insured_Vintage')
])

train = df[:train.shape[0]].to_pandas()
test = df[train.shape[0]:].to_pandas()

In [17]:
#Why just interactions with Previously_Insured variable?
    #https://www.kaggle.com/competitions/playground-series-s4e7/discussion/516475#2901376
#-> We ran all combinations of interactions on a smaller subset (500K rows) and chose the Top-3/4 based on feature importance. 
    #Any interaction with Previously_Insured seemed to work the best

# It was on top of feature importance, as seen on other topic:
    #https://www.kaggle.com/competitions/playground-series-s4e7/discussion/516860

#Feature Engeneering with high Feature Importances:
    #https://www.kaggle.com/competitions/playground-series-s4e7/discussion/517112
    #Previously_Insured_Vehicle_Damage
    #Vehicle_Age_Damage

In [18]:
# PANDAS EQUIVALENT:

# Concatenate train and test DataFrames
# df_pd = pd.concat([train_pd, test_pd], ignore_index=True)

# # Convert specific columns
# df_pd['Gender'] = df_pd['Gender'].replace({'Male': 0, 'Female': 1}).astype('int32')
# df_pd['Region_Code'] = df_pd['Region_Code'].astype(int)
# df_pd['Vehicle_Age'] = df_pd['Vehicle_Age'].replace({'< 1 Year': 0, '1-2 Year': 1, '> 2 Years': 2}).astype('int32')
# df_pd['Vehicle_Damage'] = df_pd['Vehicle_Damage'].replace({'No': 0, 'Yes': 1}).astype('int32')
# df_pd['Annual_Premium'] = df_pd['Annual_Premium'].astype(int)
# df_pd['Policy_Sales_Channel'] = df_pd['Policy_Sales_Channel'].astype(int)

# # Factorize and create new columns  -> pd.factorize() is a function in Pandas that is used to encode categorical (or object) data as integer values.
# df_pd['Previously_Insured_Annual_Premium'] = pd.factorize(df_pd['Previously_Insured'].astype(str) + df_pd['Annual_Premium'].astype(str))[0]
# df_pd['Previously_Insured_Vehicle_Age'] = pd.factorize(df_pd['Previously_Insured'].astype(str) + df_pd['Vehicle_Age'].astype(str))[0]
# df_pd['Previously_Insured_Vehicle_Damage'] = pd.factorize(df_pd['Previously_Insured'].astype(str) + df_pd['Vehicle_Damage'].astype(str))[0]
# df_pd['Previously_Insured_Vintage'] = pd.factorize(df_pd['Previously_Insured'].astype(str) + df_pd['Vintage'].astype(str))[0]

# # Split back into train and test DataFrames
# train_pd = df_pd.iloc[:len(train_pd)]
# test_pd = df_pd.iloc[len(train_pd):]

# # Reset index if needed
# train_pd.reset_index(drop=True, inplace=True)
# test_pd.reset_index(drop=True, inplace=True)

# # Optionally, print or inspect the resulting train_pd and test_pd DataFrames
# print(train_pd.head())
# print(test_pd.head())


In [19]:
#convert all cols to lower case, and replace spaces and dots with underscores
train.columns = train.columns.str.replace(' ','_').str.replace('.','_').str.lower()
test.columns = test.columns.str.replace(' ','_').str.replace('.','_').str.lower()

In [20]:
train[:2]

,id,gender,age,driving_license,region_code,previously_insured,vehicle_age,vehicle_damage,annual_premium,policy_sales_channel,vintage,response,previously_insured_annual_premium,previously_insured_vehicle_age,previously_insured_vehicle_damage,previously_insured_vintage
0,5261264,1,21,1,47,1,0,0,41835,152,122,0,0,0,0,0
1,11305325,1,20,1,46,1,0,0,27499,160,124,0,1,0,0,1


In [21]:
test[:2]

,id,gender,age,driving_license,region_code,previously_insured,vehicle_age,vehicle_damage,annual_premium,policy_sales_channel,vintage,response,previously_insured_annual_premium,previously_insured_vehicle_age,previously_insured_vehicle_damage,previously_insured_vintage
0,11504798,1,20,1,47,0,0,0,2630,160,228,0,7,3,2,302
1,11504799,0,47,1,28,0,1,1,37483,124,123,0,13873,1,1,360


In [10]:
train.shape

(500000, 16)

In [47]:
test.shape

(7669866, 16)

In [22]:
#variations of each column
print(f"- Distinct values:\n{train.apply( lambda x: x.unique().shape[0] ) } \n" )
print(f"- Variations:\n{train.apply( lambda x: x.unique() ) } " )

- Distinct values:
id                                   500000
gender                                    2
age                                      65
driving_license                           2
region_code                              53
previously_insured                        2
vehicle_age                               3
vehicle_damage                            2
annual_premium                        36848
policy_sales_channel                    135
vintage                                 290
response                                  2
previously_insured_annual_premium     61255
previously_insured_vehicle_age            6
previously_insured_vehicle_damage         4
previously_insured_vintage              580
dtype: int64 

- Variations:
id                                   [5261264, 11305325, 2426822, 5774546, 10258821, 299478, 7921513, 4557830, 3486424, 4743639, 3448...
gender                                                                                                         

# Dataset Split

In [23]:
#train w/ 500k rows
train[:2]

,id,gender,age,driving_license,region_code,previously_insured,vehicle_age,vehicle_damage,annual_premium,policy_sales_channel,vintage,response,previously_insured_annual_premium,previously_insured_vehicle_age,previously_insured_vehicle_damage,previously_insured_vintage
0,5261264,1,21,1,47,1,0,0,41835,152,122,0,0,0,0,0
1,11305325,1,20,1,46,1,0,0,27499,160,124,0,1,0,0,1


In [24]:
X = train.drop(columns="response") 
y = train.response

In [25]:
#split between train (80%) and test (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 98, stratify = y, test_size = 0.20) #stratify by y

In [26]:
# train data
print('X_train -> ',X_train.shape)
print('y_train -> ',y_train.shape)
# test data
print('X_test -> ',X_test.shape)
print('y_test -> ',y_test.shape)

X_train ->  (400000, 15)
y_train ->  (400000,)
X_test ->  (100000, 15)
y_test ->  (100000,)


In [27]:
#destroy df
df=''

# ML

In [38]:
#CatBoostClassifier

#params: instead os a tuple, create a dict with range:
search_spaces = {'iterations':       space.Integer(50, 1000), #5000 original
                'learning_rate':     space.Real(1e-5, .3, prior='log-uniform'),  # 1 × 10−5 -> 0.00001.
                'depth':             space.Integer(3, 10),
                'l2_leaf_reg':       space.Real(0.2, 10),
                'border_count':      space.Integer(32, 254),
                'learning_rate':     space.Real(0.01, 1.0),
                'scale_pos_weight':  space.Real(1.0, 10.0),
                'min_child_samples': space.Integer(1, 100),
                'subsample':         space.Real(0.05, 1.0),
                'random_strength':   space.Real(0.0, 2.0),
                'colsample_bylevel': space.Real(0.5, 1.0)}


#instantiate CatBoostClassifier
catboost_cv = CatBoostClassifier( loss_function='Logloss', 
                                  eval_metric='AUC',  
                                  task_type= 'CPU', #'GPU' -> Demands NVidea graphics cards w/ CUDA)
                                  random_seed=98)
                                  #verbose=False

#best parameters w/ CatBoostClassifier
catboost_bayes_cv = BayesSearchCV(estimator = catboost_cv,
                                  search_spaces = search_spaces,
                                  n_jobs = -1,
                                  n_iter = 50, #how many times the search is performed - default=50
                                                #n_iter trades off runtime vs quality of the solution.
                                              #Consider increasing n_points if you want to try more parameter settings in parallel.
                                  n_points = 10, #how many points are sampled in each iteration
                                                #use = 10 first time, to use parallelism max and run fast (> 10 não muda score)
                                                #the best result is with n_points = 1, but it is much slower (about 10x +)
                                  cv = 3,
                                  #Núum of models = n_iter * n_points * CV (Ex: 80*10*5 = 4000)
                                  random_state = 98,
                                  scoring = 'roc_auc',
                                  refit = True,
                                  verbose = 2,
                                  return_train_score = True )

#train
catboost_bayes_cv.fit(X_train, y_train) #vazamento do preprocessing, usar pipeline

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Fitting 3 folds for each of 10 candidates, totalling 30 fits
Fitting 3 folds for each of 10 candidates, totalling 30 fits
Fitting 3 folds for each of 10 candidates, totalling 30 fits
Fitting 3 folds for each of 10 candidates, totalling 30 fits
0:	total: 28.8ms	remaining: 23.9s
1:	total: 60.3ms	remaining: 25s
2:	total: 96.4ms	remaining: 26.6s
3:	total: 129ms	remaining: 26.5s
4:	total: 158ms	remaining: 26s
5:	total: 182ms	remaining: 24.9s
6:	total: 202ms	remaining: 23.8s
7:	total: 226ms	remaining: 23.2s
8:	total: 247ms	remaining: 22.5s
9:	total: 270ms	remaining: 22.2s
10:	total: 293ms	remaining: 21.8s
11:	total: 313ms	remaining: 21.4s
12:	total: 336ms	remaining: 21.1s
13:	total: 359ms	remaining: 20.9s
14:	total: 380ms	remaining: 20.6s
15:	total: 401ms	remaining: 20.4s
16:	total: 425ms	remaining: 20.3s
17:	total: 446ms	remaining: 20.1s
18:	total: 468ms	remaining: 20s
19:	total: 491ms	remaining: 19.9s
20:	total: 514ms	remaining: 

BayesSearchCV(cv=3,
              estimator=<catboost.core.CatBoostClassifier object at 0x0000023D5E083C20>,
              n_jobs=-1, n_points=10, random_state=98, return_train_score=True,
              scoring='roc_auc',
              search_spaces={'border_count': Integer(low=32, high=254, prior='uniform', transform='normalize'),
                             'colsample_bylevel': Real(low=0.5, high=1.0, prior='uniform', transform='normalize'),
                             'depth...
                             'learning_rate': Real(low=0.01, high=1.0, prior='uniform', transform='normalize'),
                             'min_child_samples': Integer(low=1, high=100, prior='uniform', transform='normalize'),
                             'random_strength': Real(low=0.0, high=2.0, prior='uniform', transform='normalize'),
                             'scale_pos_weight': Real(low=1.0, high=10.0, prior='uniform', transform='normalize'),
                             'subsample': Real(low=0.05, high=1.0, prior='uniform', transform='normalize')},
              verbose=2)

In [42]:
#best params
catboost_bayes_cv.best_params_

OrderedDict([('border_count', 254),
             ('colsample_bylevel', 0.9845482380514026),
             ('depth', 3),
             ('iterations', 830),
             ('l2_leaf_reg', 10.0),
             ('learning_rate', 0.4077860537261423),
             ('min_child_samples', 1),
             ('random_strength', 2.0),
             ('scale_pos_weight', 1.8468568144962254),
             ('subsample', 0.5208838452498966)])

In [43]:
#evaluate (refit=True -> its already trained on best params)
catboost_bayes_cv.score(X_test, y_test)

0.876784589284172

In [44]:
#first run: 0.8717885909532188
#second run: 0.876784589284172

In [45]:
#check overfitting:
pd.set_option('max_colwidth', 250)
pd.DataFrame(catboost_bayes_cv.cv_results_)[['rank_test_score','params','mean_train_score','mean_test_score']].sort_values('rank_test_score')[:5]

,rank_test_score,params,mean_train_score,mean_test_score
47,1,"{'border_count': 254, 'colsample_bylevel': 0.9845482380514026, 'depth': 3, 'iterations': 830, 'l2_leaf_reg': 10.0, 'learning_rate': 0.4077860537261423, 'min_child_samples': 1, 'random_strength': 2.0, 'scale_pos_weight': 1.8468568144962254, 'subsa...",0.892,0.876
42,2,"{'border_count': 176, 'colsample_bylevel': 0.5540483258861371, 'depth': 4, 'iterations': 352, 'l2_leaf_reg': 10.0, 'learning_rate': 0.31036690154596763, 'min_child_samples': 1, 'random_strength': 1.2579277679534602, 'scale_pos_weight': 3.74190520...",0.886,0.875
17,3,"{'border_count': 130, 'colsample_bylevel': 0.506260064055757, 'depth': 3, 'iterations': 1000, 'l2_leaf_reg': 10.0, 'learning_rate': 0.36539166170485465, 'min_child_samples': 26, 'random_strength': 1.973701027130659, 'scale_pos_weight': 5.30981718...",0.891,0.875
6,4,"{'border_count': 124, 'colsample_bylevel': 0.9013829455566414, 'depth': 4, 'iterations': 661, 'l2_leaf_reg': 3.2964564345030123, 'learning_rate': 0.23902748837074625, 'min_child_samples': 27, 'random_strength': 0.7069218334491335, 'scale_pos_weig...",0.889,0.875
40,5,"{'border_count': 236, 'colsample_bylevel': 0.9944563142206998, 'depth': 5, 'iterations': 494, 'l2_leaf_reg': 9.29060731962691, 'learning_rate': 0.2821257884947522, 'min_child_samples': 23, 'random_strength': 0.023940024048759254, 'scale_pos_weigh...",0.897,0.874


In [ ]:
#if params are on their border, there can be impovements.

In [ ]:
#Lets submit by now.

# Kaggle Test Prediction

In [49]:
print(test.shape) #7 MI rows!
test[:3]

(7669866, 16)


,id,gender,age,driving_license,region_code,previously_insured,vehicle_age,vehicle_damage,annual_premium,policy_sales_channel,vintage,response,previously_insured_annual_premium,previously_insured_vehicle_age,previously_insured_vehicle_damage,previously_insured_vintage
0,11504798,1,20,1,47,0,0,0,2630,160,228,0,7,3,2,302
1,11504799,0,47,1,28,0,1,1,37483,124,123,0,13873,1,1,360
2,11504800,0,47,1,43,0,1,1,2630,26,271,0,7,1,1,340


In [50]:
#drop response col
test.drop(columns='response', inplace=True)

In [51]:
#predict on prod data:
y_hat_prod = catboost_bayes_cv.predict_proba(test)[:,1]

In [57]:
test_id = test['id'].copy()

In [52]:
#Predictions done:
print(y_hat_prod.shape)
y_hat_prod[:5]

(7669866,)


array([0.00579 , 0.531259, 0.310568, 0.00013 , 0.087544])

In [54]:
test

,id,gender,age,driving_license,region_code,previously_insured,vehicle_age,vehicle_damage,annual_premium,policy_sales_channel,vintage,previously_insured_annual_premium,previously_insured_vehicle_age,previously_insured_vehicle_damage,previously_insured_vintage
0,11504798,1,20,1,47,0,0,0,2630,160,228,7,3,2,302
1,11504799,0,47,1,28,0,1,1,37483,124,123,13873,1,1,360
2,11504800,0,47,1,43,0,1,1,2630,26,271,7,1,1,340
3,11504801,1,22,1,47,1,0,0,24502,152,115,16218,0,0,179
4,11504802,0,51,1,19,0,1,0,34115,124,148,17148,1,2,401
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7669861,19174659,0,57,1,28,0,1,1,51661,124,109,6226,1,1,208
7669862,19174660,0,28,1,50,1,0,0,25651,152,184,21685,0,0,249
7669863,19174661,0,47,1,33,1,1,0,2630,138,63,22,4,0,445
7669864,19174662,0,30,1,28,0,0,1,38866,124,119,17251,3,1,378


In [58]:
pd.Series(test_id, name='id')

0          11504798
1          11504799
2          11504800
3          11504801
4          11504802
             ...   
7669861    19174659
7669862    19174660
7669863    19174661
7669864    19174662
7669865    19174663
Name: id, Length: 7669866, dtype: int64

In [59]:
# Prepare for submission: id + predictions:
# Create final dataset
df_sub = pd.concat([ pd.Series(test_id, name='id') , pd.Series(y_hat_prod, name='Target') ], axis=1)
df_sub[:3]

,id,Target
0,11504798,0.006
1,11504799,0.531
2,11504800,0.311


In [60]:
# Export to submit on Kaggle
df_sub.to_parquet('..\\data\\v8\\submission_v8.parquet', engine='pyarrow')